In [ ]:
!wget https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-v3-stable-darwin-amd64.zip
!unzip ngrok-v3-stable-darwin-amd64.zip

--2023-11-02 17:44:57--  https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-v3-stable-darwin-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 54.237.133.81, 52.202.168.65, 18.205.222.128, ...
Connecting to bin.equinox.io (bin.equinox.io)|54.237.133.81|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9393876 (9.0M) [application/octet-stream]
Saving to: ‘ngrok-v3-stable-darwin-amd64.zip.1’

ngrok-v3-stable-dar 100%[===================>]   8.96M  16.2MB/s    in 0.6s    

2023-11-02 17:44:58 (16.2 MB/s) - ‘ngrok-v3-stable-darwin-amd64.zip.1’ saved [9393876/9393876]

Archive:  ngrok-v3-stable-darwin-amd64.zip
replace ngrok? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
!./ngrok config add-authtoken 1tHaGIL1YdMNniMQKsAzqSgsiUb_7AgZfgwYUTdCzFGva4KE8

In [1]:
from Train import train
from TrainPPO import train_ppo
from webapp2048.WebApp import run_async
run_async()
train_ppo(
    epochs=1000,
    mcts_iterations=300,
    batch_size=1000,
    temperature=1.0,
    window_size=5
)

2023-11-05 16:34:24.846707: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Public URL: http://0.0.0.0:5001
decayed_entropy=0.009999010000000001
decayed_lr=0.000999901
PPO Loss: 119204.5625, Value Loss: 113298.2031
average_reward=5.86425406899516
decayed_entropy=0.00999802
decayed_lr=0.000999802
PPO Loss: 3211.3274, Value Loss: 99.2392
average_reward=5.539372279148294
decayed_entropy=0.00999703
decayed_lr=0.000999703
PPO Loss: 1973.3591, Value Loss: 39.4603
average_reward=6.578218037782484
decayed_entropy=0.009996040000000001
decayed_lr=0.000999604
PPO Loss: 3924.1904, Value Loss: 48.2280
average_reward=5.65639397422368
decayed_entropy=0.00999505
decayed_lr=0.0009995050000000001
PPO Loss: 1506.3004, Value Loss: 46.9323
average_reward=6.7044298433910114
decayed_entropy=0.009994059999999999
decayed_lr=0.000999406
PPO Loss: 1598.3823, Value Loss: 52.0997
average_reward=6.472481354971381
decayed_entropy=0.00999307
decayed_lr=0.000999307
PPO Loss: 2587.9604, Value Loss: 23.0111
average_reward=6.638559681125054
decayed_entropy=0.00999208
decayed_lr=0.000999208
PPO L

KeyboardInterrupt: 

1440

In [1]:
%autoawait on

from VectorizedTrainPPO import train_vectorized_ppo
from webapp2048.WebApp import run_async
run_async()
await train_vectorized_ppo(
    epochs=10000,
    batch_size=1000,
    temperature=1.0,
    num_envs=1
)

2023-11-04 16:21:53.435742: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Public URL: http://0.0.0.0:5001
updward_max_board_tile=2
updward_max_board_tile=4
updward_max_board_tile=8
updward_max_board_tile=16
updward_max_board_tile=32
updward_max_board_tile=64
updward_max_board_tile=128
updward_max_board_tile=256
updward_max_board_tile=512
updward_max_board_tile=1024
updward_max_board_tile=2048
updward_max_board_tile=4096
updward_max_board_tile=8192
updward_max_board_tile=16384
updward_max_board_tile=32768
updward_max_board_tile=65536
updward_max_board_tile=131072
downward_max_board_tile=2.0
downward_max_board_tile=1.0
updward_max_board_tile=8
updward_max_board_tile=16
updward_max_board_tile=32
updward_max_board_tile=64
updward_max_board_tile=128
updward_max_board_tile=256
updward_max_board_tile=512
updward_max_board_tile=1024
updward_max_board_tile=2048
updward_max_board_tile=4096
updward_max_board_tile=8192
updward_max_board_tile=16384
updward_max_board_tile=32768
updward_max_board_tile=65536
updward_max_board_tile=131072
downward_max_board_tile=1.0
updward_

ValueError: cannot reshape array of size 368 into shape (1,4,4,5)

In [1]:
def is_next_to_lower_tile(board, tile_value):
    """
    Check if a tile with the given value is adjacent to a tile with half its value.
    """
    for i in range(4):
        for j in range(4):
            if board[i][j] == tile_value:
                # Check adjacent cells for half the tile value
                adjacent_cells = [(i-1, j), (i+1, j), (i, j-1), (i, j+1)]
                for cell in adjacent_cells:
                    x, y = cell
                    if 0 <= x < 4 and 0 <= y < 4 and board[x][y] == tile_value / 2:
                        return True
    return False

# Test the helper function
is_next_to_lower_tile(test_board, 4)  # Expect True because 4 is next to 2 on the board.


NameError: name 'test_board' is not defined

In [ ]:
def calculate_reward(board):
    # Calculate the maximum tile and number of iterations
    max_tile = max(max(row) for row in board)
    iterations = int(max_tile**0.5)

    reward = 0

    for i in range(1, iterations+1):
        tile_value = 2**i
        tile_count = sum(row.count(tile_value) for row in board)

        # Check for adjacent tiles with the same value
        if are_adjacent(board, tile_value):
            multiplier = 1.75
        # Check for tile adjacent to the next lower value tile
        elif is_next_to_lower_tile(board, tile_value):
            multiplier = 1.5
        # Check for 2 or more non-adjacent tiles
        elif tile_count >= 2:
            multiplier = 1.25
        # Check for single tile
        elif tile_count == 1:
            multiplier = 1.0
        # No tile present
        else:
            multiplier = 0.0

        reward += multiplier * tile_value

    return reward

# Test the main function with the test board
calculate_reward(test_board)


In [ ]:
calculate_reward([
    [4,0,0,0],
    [0,0,0,0],
    [0,0,0,0],
    [0,0,0,0],
])